In [1]:
# =============================
# library
# =============================
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import polars as pl
from sklearn.preprocessing import StandardScaler
import glob
from tqdm import tqdm

In [2]:
# =============================
# constant
# =============================
TRAIN_PATH = Path("../storage/leap/data/train.csv")
TEST_PATH = Path("../storage/leap/data/test.csv")
OUTPUT_DIR = Path("../storage/leap/output")
DATA_DIR = Path("../storage/leap/data")
TRAIN_DATA1_DIR = Path("../storage/leap/data/train_0001")

In [3]:
# =============================
# settings
# =============================
fe = "121"
fe_dir = OUTPUT_DIR / "fe" / f"fe{fe}"
fe_dir.mkdir(parents=True, exist_ok=True)
fe_save_dir = fe_dir / "save"
fe_save_dir.mkdir(parents=True, exist_ok=True)

In [4]:
# =============================
# columns
# =============================
state_t = [f'state_t_{i}' for i in range(60)]
state_q0001 = [f'state_q0001_{i}' for i in range(60)]
state_q0002 = [f'state_q0002_{i}' for i in range(60)]
state_q0003 = [f'state_q0003_{i}' for i in range(60)]
state_u = [f'state_u_{i}' for i in range(60)]
state_v = [f'state_v_{i}' for i in range(60)]
other = ['state_ps', 'pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX',
       'pbuf_TAUX', 'pbuf_TAUY', 'pbuf_COSZRS', 'cam_in_ALDIF', 'cam_in_ALDIR',
       'cam_in_ASDIF', 'cam_in_ASDIR', 'cam_in_LWUP', 'cam_in_ICEFRAC',
       'cam_in_LANDFRAC', 'cam_in_OCNFRAC', 'cam_in_SNOWHLAND']
pbuf_ozone = [f'pbuf_ozone_{i}' for i in range(60)]
pbuf_CH4 = [f'pbuf_CH4_{i}' for i in range(60)]
pbuf_N2O = [f'pbuf_N2O_{i}' for i in range(60)]
cols_list = [state_t,state_q0001,state_q0002,state_q0003,
             state_u,state_v,pbuf_ozone,pbuf_CH4,pbuf_N2O,other]

In [5]:
# =============================
# target
# =============================
ptend_t = [f'ptend_t_{i}' for i in range(60)]
ptend_q0001 = [f'ptend_q0001_{i}' for i in range(60)]
ptend_q0002 = [f'ptend_q0002_{i}' for i in range(60)]
ptend_q0003 = [f'ptend_q0003_{i}' for i in range(60)]
ptend_u = [f'ptend_u_{i}' for i in range(60)]
ptend_v = [f'ptend_v_{i}' for i in range(60)]
other_target= ['cam_out_NETSW', 'cam_out_FLWDS', 'cam_out_PRECSC', 'cam_out_PRECC', 'cam_out_SOLS', 'cam_out_SOLL', 'cam_out_SOLSD', 'cam_out_SOLLD']
target_list = [ptend_t,ptend_q0001, ptend_q0002,ptend_q0003,ptend_u,ptend_v,other_target]

In [6]:
# =============================
# main
# =============================
# 2分の1に間引く
df_all = []
for i in tqdm(range(2, 13)):
    df = pl.read_parquet(f"{TRAIN_DATA1_DIR}/train_0001_{i}.parquet")
    print(i, df.shape[0])
    df = df.with_columns(pl.Series("group", np.arange(len(df))))
    df = df.with_columns((pl.col("group") // 384).alias("group"))
    df = df.with_columns((pl.col("group") % 2).alias("group"))
    df = df.filter(pl.col("group") == 0).drop("group")
    df_all.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

2 774144


  9%|▉         | 1/11 [00:00<00:09,  1.05it/s]

3 857088


 18%|█▊        | 2/11 [00:01<00:08,  1.04it/s]

...

 82%|████████▏ | 9/11 [00:08<00:01,  1.12it/s]

11 829440


 91%|█████████ | 10/11 [00:09<00:00,  1.12it/s]

12 857088


100%|██████████| 11/11 [00:09<00:00,  1.10it/s]


In [7]:
df_all = pl.concat(df_all)

In [8]:
print(df_all.shape)

(4617216, 925)


In [9]:
df_all.write_parquet(fe_dir / "train_0001_all_2_concat.parquet")